### Evaluating Jazz album prices on Discogs.com
### Dataprep
- program: 02_dataprep.ipynb
- purpose: readin .csv output from webscrape program (discog_alt.ipynb)
- date: jan 16, 2021
- created by: chris chan

- final clean dataframe: ../data/clean/df_clean.csv

**1. Read-in discogs csv files**

In [14]:
import pandas as pd 
import glob
import os
import numpy as np

# get all .csv files in data directory
filenames=glob.glob(r"../data/*.csv")
#print(filenames)

li=[]
df=pd.DataFrame()

# readin each csv and place into list
for i in filenames:
    data = pd.read_csv(i) 
    li.append(data)
    
# append data together    
df=pd.concat(li,axis=0,ignore_index=True)
df.shape

(4079, 14)

In [15]:
df.head()

,Artist_Album,Label,Country,Format,Genre,Release_Date,Style,Rate_Haves_Wants,Media_Condition,Sleeve_Condition,Seller_Rating,Recorded_at,Pressed_by,Price
0,Miles Davis – Volume 2,Blue Note – BLP 1502,US,"Vinyl, LP, Compilation, Remastered, Mono",Jazz,Feb-56,"Hard Bop, Bop","Rated 4.32 of 25 votes (219 have, 302 want)",Very Good Plus (VG+),Sleeve: Very Good (VG),98.80%,WOR Studios,Plastylite,"$1,100.00"
1,Curtis Fuller – The Opener,Blue Note – BLP 1567,US,"Vinyl, LP, Album, Mono",Jazz,1957,Hard Bop,"Rated 4.50 of 16 votes (68 have, 385 want)",Very Good (VG),Sleeve: Very Good (VG),100.00%,"Van Gelder Studio, Hackensack, New Jersey",NaN,"$2,100.00"
2,The Jazz Messengers* – At The Cafe Bohemia Vol...,Blue Note – BLP 1508,US,"Vinyl, LP, Album, Mono",Jazz,May-56,Hard Bop,"Rated 4.62 of 29 votes (205 have, 409 want)",Near Mint (NM or M-),Sleeve: Very Good Plus (VG+),98.80%,Café Bohemia,Plastylite,$770.00
3,Cliff Jordan* & John Gilmore – Blowing In From...,Blue Note – BLP 1549,US,"Vinyl, LP, Album, Mono",Jazz,May-57,Hard Bop,"Rated 4.76 of 17 votes (83 have, 493 want)",Very Good Plus (VG+),Sleeve: Very Good (VG),97.80%,"Van Gelder Studio, Hackensack, New Jersey",Plastylite,"$1,650.00"
4,Cannonball Adderley – Somethin' Else,"Blue Note – BLP 1595, Blue Note – 1595",US,"Vinyl, LP, Album, Mono",Jazz,May-58,"Bop, Hard Bop","Rated 4.70 of 194 votes (1085 have, 1716 want)",Very Good Plus (VG+),Sleeve: Very Good Plus (VG+),NaN,NaN,Plastylite,"$1,099.00"


**2. Data cleaning:**
- arist_album:
    split artist / album by (-)
- label
    split by (-)
- format
    split by (,)
- release date
    get year - split by (-), otherwise year
rates_have_wants
    - split by ( first
- rating
    - split rates_haves_want (2nd)
- votes
    - split rates_haves_wants (4th)
- haves

    - split rates_haves_wants : after ( get 1st
- wants
    - split rates_haves_wants : after ( get 3rd

In [16]:
'''
 Perform cleaning of vars.
 Note: a few 'dashes' are em dashes. These were copied into the condition to have the variables split
'''

# Artist, Album
df[['artist', 'album']] = df['Artist_Album'].str.split('–', n=1, expand=True)

# Record Label
df[['reclabel','_reclabel']] = df['Label'].str.split('–',n=1,expand=True)

# Record release yeaer
df['recyear'] = df['Release_Date'].str[-2:] #  = sample_str[-1]

# Rating haves wants votes
df[['_rating','_haveswants']] = df['Rate_Haves_Wants'].str.split('(', 1,expand=True)

# Split haves, wants
df[['_haves','_wants']] = df['_haveswants'].str.split(',', 1,expand=True)
df['_wants'] = df['_wants'].str[:-1]

# Sleeve condition
df['sleeve_condition']=df['Sleeve_Condition'].str.split(':',1,expand=True)[1]
df['sleeve_cond_abbr']=df['Sleeve_Condition'].str.split('(',1,expand=True)[1]
df['sleeve_cond_abbr']=df['sleeve_cond_abbr'].str.replace(')','')

# Media condition
df['media_cond_abbr']=df['Media_Condition'].str.split('(',1,expand=True)[1]
df['media_cond_abbr']=df['media_cond_abbr'].str.replace(')','')

# Price
df['_price'] = df['Price'].map(lambda x: x.lstrip('$,').rstrip('aAbBcC,'))

# Seller rating
#df['_seller_rating'] = df['Seller_Rating'].map(lambda x: x.strip('%'))

df['seller_rating'] = df['Seller_Rating'].astype(str).str.replace('%', '').astype(float) / 100

In [17]:
df.head()

,Artist_Album,Label,Country,Format,Genre,Release_Date,Style,Rate_Haves_Wants,Media_Condition,Sleeve_Condition,...,recyear,_rating,_haveswants,_haves,_wants,sleeve_condition,sleeve_cond_abbr,media_cond_abbr,_price,seller_rating
0,Miles Davis – Volume 2,Blue Note – BLP 1502,US,"Vinyl, LP, Compilation, Remastered, Mono",Jazz,Feb-56,"Hard Bop, Bop","Rated 4.32 of 25 votes (219 have, 302 want)",Very Good Plus (VG+),Sleeve: Very Good (VG),...,56,Rated 4.32 of 25 votes,"219 have, 302 want)",219 have,302 want,Very Good (VG),VG,VG+,"1,100.00",0.988
1,Curtis Fuller – The Opener,Blue Note – BLP 1567,US,"Vinyl, LP, Album, Mono",Jazz,1957,Hard Bop,"Rated 4.50 of 16 votes (68 have, 385 want)",Very Good (VG),Sleeve: Very Good (VG),...,57,Rated 4.50 of 16 votes,"68 have, 385 want)",68 have,385 want,Very Good (VG),VG,VG,"2,100.00",1.000
2,The Jazz Messengers* – At The Cafe Bohemia Vol...,Blue Note – BLP 1508,US,"Vinyl, LP, Album, Mono",Jazz,May-56,Hard Bop,"Rated 4.62 of 29 votes (205 have, 409 want)",Near Mint (NM or M-),Sleeve: Very Good Plus (VG+),...,56,Rated 4.62 of 29 votes,"205 have, 409 want)",205 have,409 want,Very Good Plus (VG+),VG+,NM or M-,770.00,0.988
3,Cliff Jordan* & John Gilmore – Blowing In From...,Blue Note – BLP 1549,US,"Vinyl, LP, Album, Mono",Jazz,May-57,Hard Bop,"Rated 4.76 of 17 votes (83 have, 493 want)",Very Good Plus (VG+),Sleeve: Very Good (VG),...,57,Rated 4.76 of 17 votes,"83 have, 493 want)",83 have,493 want,Very Good (VG),VG,VG+,"1,650.00",0.978
4,Cannonball Adderley – Somethin' Else,"Blue Note – BLP 1595, Blue Note – 1595",US,"Vinyl, LP, Album, Mono",Jazz,May-58,"Bop, Hard Bop","Rated 4.70 of 194 votes (1085 have, 1716 want)",Very Good Plus (VG+),Sleeve: Very Good Plus (VG+),...,58,Rated 4.70 of 194 votes,"1085 have, 1716 want)",1085 have,1716 want,Very Good Plus (VG+),VG+,VG+,"1,099.00",NaN


**2b. More Data Cleaning**

In [18]:
# Clean up vars:

# Ratings
df['rating']=df['_rating'].str.split(' ',4,expand=True)[1].str.strip()
df['rating'].replace({'Rating': np.nan},inplace=True)
df['rating']=df['rating'].astype(float)

# Votes
df['votes']=df['_rating'].str.split(' ',4,expand=True)[3].str.strip()
df['votes']=pd.to_numeric(df['votes'], errors='coerce')

# Haves / Wants
df['haves']=df['_haves'].str.split(' ',1,expand=True)[0].astype(int)
df['wants']=df['_wants'].str.split(' ',2,expand=True)[1].astype(int)

# Price
df['price'] = df['_price'].str.replace(',', '').astype(float)

#print("rating:",testrating,"votes:",testvotes,"haves:",testhaves,"wants",testwants)

In [19]:
df.head()

,Artist_Album,Label,Country,Format,Genre,Release_Date,Style,Rate_Haves_Wants,Media_Condition,Sleeve_Condition,...,sleeve_condition,sleeve_cond_abbr,media_cond_abbr,_price,seller_rating,rating,votes,haves,wants,price
0,Miles Davis – Volume 2,Blue Note – BLP 1502,US,"Vinyl, LP, Compilation, Remastered, Mono",Jazz,Feb-56,"Hard Bop, Bop","Rated 4.32 of 25 votes (219 have, 302 want)",Very Good Plus (VG+),Sleeve: Very Good (VG),...,Very Good (VG),VG,VG+,"1,100.00",0.988,4.32,25.0,219,302,1100.0
1,Curtis Fuller – The Opener,Blue Note – BLP 1567,US,"Vinyl, LP, Album, Mono",Jazz,1957,Hard Bop,"Rated 4.50 of 16 votes (68 have, 385 want)",Very Good (VG),Sleeve: Very Good (VG),...,Very Good (VG),VG,VG,"2,100.00",1.000,4.50,16.0,68,385,2100.0
2,The Jazz Messengers* – At The Cafe Bohemia Vol...,Blue Note – BLP 1508,US,"Vinyl, LP, Album, Mono",Jazz,May-56,Hard Bop,"Rated 4.62 of 29 votes (205 have, 409 want)",Near Mint (NM or M-),Sleeve: Very Good Plus (VG+),...,Very Good Plus (VG+),VG+,NM or M-,770.00,0.988,4.62,29.0,205,409,770.0
3,Cliff Jordan* & John Gilmore – Blowing In From...,Blue Note – BLP 1549,US,"Vinyl, LP, Album, Mono",Jazz,May-57,Hard Bop,"Rated 4.76 of 17 votes (83 have, 493 want)",Very Good Plus (VG+),Sleeve: Very Good (VG),...,Very Good (VG),VG,VG+,"1,650.00",0.978,4.76,17.0,83,493,1650.0
4,Cannonball Adderley – Somethin' Else,"Blue Note – BLP 1595, Blue Note – 1595",US,"Vinyl, LP, Album, Mono",Jazz,May-58,"Bop, Hard Bop","Rated 4.70 of 194 votes (1085 have, 1716 want)",Very Good Plus (VG+),Sleeve: Very Good Plus (VG+),...,Very Good Plus (VG+),VG+,VG+,"1,099.00",NaN,4.70,194.0,1085,1716,1099.0


**3. Diagnostics**

In [20]:
# check values of vars
#df.dtypes
df['Style'].value_counts(ascending=False,dropna=False)

Hard Bop                                                    2151
Bop, Hard Bop                                                415
Hard Bop, Modal                                              259
Hard Bop, Soul-Jazz                                          166
Hard Bop, Post Bop                                           147
                                                            ... 
Avant-garde Jazz, Hard Bop, Modal                              1
Bop, Hard Bop, Afrobeat, Swing                                 1
Bossa Nova, Hard Bop, Latin Jazz                               1
Hard Bop, Fusion                                               1
Hard Bop, Latin Jazz, Big Band, Poetry, Avant-garde Jazz       1
Name: Style, Length: 122, dtype: int64

In [21]:
df.describe()

,seller_rating,rating,votes,haves,wants,price
count,4066.000000,3884.000000,3884.000000,4079.000000,4079.000000,4079.000000
mean,0.992452,4.440935,22.474768,189.257416,204.910762,134.473366
std,0.025975,0.380680,29.645899,249.913455,280.008017,198.354186
min,0.571000,1.000000,1.000000,0.000000,0.000000,40.000000
25%,0.995000,4.260000,5.000000,40.000000,59.000000,50.000000
50%,0.999000,4.500000,12.000000,95.000000,122.000000,75.000000
75%,1.000000,4.670000,27.000000,238.000000,256.000000,140.000000
max,1.000000,5.000000,209.000000,1942.000000,3679.000000,5000.000000


**4. Drop unnecessary variables**

In [22]:
# drop vars with _: and unnecessary vars
drops = df.columns[df.columns.str.startswith('_')]
df.drop(drops, axis=1, inplace=True)
df.drop(['Rate_Haves_Wants','sleeve_condition'], axis=1,inplace=True)
df.shape

(4079, 25)

In [23]:
df.head()

,Artist_Album,Label,Country,Format,Genre,Release_Date,Style,Media_Condition,Sleeve_Condition,Seller_Rating,...,reclabel,recyear,sleeve_cond_abbr,media_cond_abbr,seller_rating,rating,votes,haves,wants,price
0,Miles Davis – Volume 2,Blue Note – BLP 1502,US,"Vinyl, LP, Compilation, Remastered, Mono",Jazz,Feb-56,"Hard Bop, Bop",Very Good Plus (VG+),Sleeve: Very Good (VG),98.80%,...,Blue Note,56,VG,VG+,0.988,4.32,25.0,219,302,1100.0
1,Curtis Fuller – The Opener,Blue Note – BLP 1567,US,"Vinyl, LP, Album, Mono",Jazz,1957,Hard Bop,Very Good (VG),Sleeve: Very Good (VG),100.00%,...,Blue Note,57,VG,VG,1.000,4.50,16.0,68,385,2100.0
2,The Jazz Messengers* – At The Cafe Bohemia Vol...,Blue Note – BLP 1508,US,"Vinyl, LP, Album, Mono",Jazz,May-56,Hard Bop,Near Mint (NM or M-),Sleeve: Very Good Plus (VG+),98.80%,...,Blue Note,56,VG+,NM or M-,0.988,4.62,29.0,205,409,770.0
3,Cliff Jordan* & John Gilmore – Blowing In From...,Blue Note – BLP 1549,US,"Vinyl, LP, Album, Mono",Jazz,May-57,Hard Bop,Very Good Plus (VG+),Sleeve: Very Good (VG),97.80%,...,Blue Note,57,VG,VG+,0.978,4.76,17.0,83,493,1650.0
4,Cannonball Adderley – Somethin' Else,"Blue Note – BLP 1595, Blue Note – 1595",US,"Vinyl, LP, Album, Mono",Jazz,May-58,"Bop, Hard Bop",Very Good Plus (VG+),Sleeve: Very Good Plus (VG+),NaN,...,Blue Note,58,VG+,VG+,NaN,4.70,194.0,1085,1716,1099.0


**5. Save Dataframe for analysis**

In [26]:
df.to_csv(r'../data/clean/df_clean.csv', index = False, header=True)
print(df)

                                           Artist_Album  \
0                                Miles Davis – Volume 2   
1                            Curtis Fuller – The Opener   
2     The Jazz Messengers* – At The Cafe Bohemia Vol...   
3     Cliff Jordan* & John Gilmore – Blowing In From...   
4                  Cannonball Adderley – Somethin' Else   
...                                                 ...   
4074                          Blue Mitchell – Heads Up!   
4075                         Mose Allison – Local Color   
4076              The Jaki Byard Quartet – Live! Vol. 2   
4077                Miles Davis Quintet* – Miles Smiles   
4078  The Dave Brubeck Quartet – Time Further Out (M...   

                                       Label Country  \
0                       Blue Note – BLP 1502      US   
1                       Blue Note – BLP 1567      US   
2                       Blue Note – BLP 1508      US   
3                       Blue Note – BLP 1549      US   
4     Blue 